In [1]:
%matplotlib inline


# Classification of text documents using sparse features


This is an example showing how scikit-learn can be used to classify documents
by topics using a bag-of-words approach. This example uses a scipy.sparse
matrix to store the features and demonstrates various classifiers that can
efficiently handle sparse matrices.

The dataset used in this example is the 20 newsgroups dataset. It will be
automatically downloaded, then cached.

The bar plot indicates the accuracy, training time (normalized) and test time
(normalized) of each classifier.




In [10]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.model_selection import train_test_split

import pandas as pd
import re
import jieba

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# parse commandline arguments
op = OptionParser()
op.add_option("--report",
              action="store_true", dest="print_report",
              help="Print a detailed classification report.")
op.add_option("--chi2_select",
              action="store", type="int", dest="select_chi2",
              help="Select some number of features using a chi-squared test")
op.add_option("--confusion_matrix",
              action="store_true", dest="print_cm",
              help="Print the confusion matrix.")
op.add_option("--top10",
              action="store_true", dest="print_top10",
              help="Print ten most discriminative terms per class"
                   " for every classifier.")
op.add_option("--all_categories",
              action="store_true", dest="all_categories",
              help="Whether to use all categories or not.")
op.add_option("--use_hashing",
              action="store_true",
              help="Use a hashing vectorizer.")
op.add_option("--n_features",
              action="store", type=int, default=2 ** 16,
              help="n_features when using the hashing vectorizer.")
op.add_option("--filtered",
              action="store_true",
              help="Remove newsgroup information that is easily overfit: "
                   "headers, signatures, and quoting.")

def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

print(__doc__)
op.print_help()



model_file = '../input/'

# 加载数据
def load_dataset():
    train = pd.read_csv(model_file + 'train_first.csv')
    test  = pd.read_csv(model_file + 'predict_first.csv')

    def clean_str(stri):
        stri = re.sub(r'[a-zA-Z0-9]+', '', stri)
        cut_str = jieba.cut(stri.strip())
        return ' '.join(cut_str)

    train['Discuss'] = train['Discuss'].apply(lambda x : clean_str(x))
    test['Discuss'] = test['Discuss'].apply(lambda x : clean_str(x))

    X = []
    for discuss in train['Discuss'].values:
        X.append(discuss)
    y = train['Score'].values

    T = []
    for discuss in test['Discuss'].values:
        T.append(discuss)

    return train[['Id']], test[['Id']], X, y, T

Automatically created module for IPython interactive environment
Usage: __main__.py [options]

Options:
  -h, --help            show this help message and exit
  --report              Print a detailed classification report.
  --chi2_select=SELECT_CHI2
                        Select some number of features using a chi-squared
                        test
  --confusion_matrix    Print the confusion matrix.
  --top10               Print ten most discriminative terms per class for
                        every classifier.
  --all_categories      Whether to use all categories or not.
  --use_hashing         Use a hashing vectorizer.
  --n_features=N_FEATURES
                        n_features when using the hashing vectorizer.
  --filtered            Remove newsgroup information that is easily overfit:
                        headers, signatures, and quoting.


In [2]:
train_id, test_id, X, y, T = load_dataset()

Building prefix dict from the default dictionary ...
2018-02-22 23:09:06,741 DEBUG Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
2018-02-22 23:09:06,749 DEBUG Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.627 seconds.
2018-02-22 23:09:08,373 DEBUG Loading model cost 1.627 seconds.
Prefix dict has been built succesfully.
2018-02-22 23:09:08,377 DEBUG Prefix dict has been built succesfully.


In [3]:
# Extracting features from the training data using a sparse vectorizer
def stop_words():
    stop_word = []
    stop_words_path = '../input/stop_word.txt'
    with open(stop_words_path, encoding='utf-8') as f:
        for line in f.readlines():
            stop_word.append(line.strip())
    return stop_word

print(stop_words()[0:10])

['\ufeff,', '?', '、', '。', '“', '”', '《', '》', '！', '，']


In [6]:
print('using TfidfVectorizer...')
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words=stop_words())

X_train = vectorizer.fit_transform(X)
print("n_samples: %d, n_features: %d" % X_train.shape)
print()

using TfidfVectorizer...
n_samples: 100000, n_features: 81052



c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [7]:
print("Extracting features from the test data using the same vectorizer")
X_test = vectorizer.transform(T)
print("n_samples: %d, n_features: %d" % X_test.shape)
print()

Extracting features from the test data using the same vectorizer
n_samples: 30000, n_features: 81052



c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [9]:
feature_names = vectorizer.get_feature_names()
if feature_names:
    feature_names = np.asarray(feature_names)
    
print(feature_names[0:10])

['__' '___' '____' '一一' '一丁点' '一万' '一万七千多' '一万个' '一万五千' '一万八千']


In [17]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

In [23]:
def benchmark(clf, X, y, X_, y_, T, target_names = ['label_1', 'label_2', 'label_3', 'label_4', 'label_5']):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X, y)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    
    t0 = time()
    pred = clf.predict(X_)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    
    score = np.sqrt(metrics.mean_squared_error(pred, y_))
    score = 1 / (1 + score)
    print("accuracy:   %0.3f" % score)
    
    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))
        
        # 利用这里的top10 的特征 作为关键词
        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
        print()
    
    if opts.print_report:
        print("classification report:")
        print(metrics.classification_report(y_, pred,
                                            target_names=target_names))
        
    if opts.print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time
    

In [14]:
X, X_, y, y_ = train_test_split(X_train, y, test_size=0.3, random_state=0)

ValueError: Found input variables with inconsistent numbers of samples: [100000, 70000]

In [24]:
opts.print_top10 = True
opts.print_report = True
opts.print_cm = True

results = []
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (Perceptron(n_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf, X, y, X_, y_, X_test))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='lsqr',
        tol=0.01)


c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\sklearn\linear_model\ridge.py:311: UserWarning: In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.
  warnings.warn("In Ridge, only 'sag' solver can currently fit the "


train time: 7.030s
test time:  0.013s
accuracy:   0.548
dimensionality: 81052
density: 0.842373
top 10 keywords per class:
label_1: 评说 一坑 什卵用 烂死 差差 坑钱 上当 极差 退票 差评
label_2: 泥娃娃 奇差 蝗虫 宽余 水太脏 乐观 恶心 小曲 超低 买点儿
label_3: 老太 费钱 请问 不过如此 唯二 冷清 失去 遮阳 海神庙 江湾
label_4: 我俩 健步 北京市 十渡 不足之处 吴越 晚饭 延庆 梁柱 说不定
label_5: 贴心 可谓 药王山 尽收眼底 限时 带领 云洞 咸亨 创造 军大衣

classification report:
             precision    recall  f1-score   support

    label_1       0.38      0.04      0.08       181
    label_2       0.62      0.02      0.03       300
    label_3       0.38      0.11      0.17      2858
    label_4       0.47      0.29      0.36      8826
    label_5       0.67      0.88      0.76     17835

avg / total       0.58      0.62      0.57     30000

confusion matrix:
[[    8     0    29    58    86]
 [    3     5    56    80   156]
 [    5     2   315   832  1704]
 [    3     0   278  2595  5950]
 [    2     1   162  1953 15717]]

Perceptron
_________________________________________________________________________

MemoryError: 